# amap.com 搜索 api/search：包成函数
- A. 步步学的代码见
 [research_amap_search_NewMedia_cities_learning.ipynb](research_amap_search_NewMedia_cities_learning.ipynb)
- B. 集成一步到位的代码见
 [research_amap_search_NewMedia_cities_onestop.ipynb](research_amap_search_NewMedia_cities_onestop.ipynb)

## 务必自己练习包成函数的能力
分析：
- 需要不断使用同一段代码
- 每次使用之关键词可能不同
- 每次使用之地区代码可能不同
- 每次使用之API key可能不同
- 返回的数据最好可能用dataframe 格式返回准备继续加工处理

函数输入：
- 关键词
- 地区代码
- API key

函数输出/返回：
- dataframe
- amap_api_search_by_keyword_city_code(kw=关键词, cc=地区代码, api_key=API key)
- amap_api_search_by_keyword_city_code(kw='植物园', cc='440000', api_key='你自己申請到的key')


In [1]:
import requests #用於API HTTP requests
import pandas as pd


def amap_api_search_by_keyword_city_code(kw, cc, api_key):

    parameters = {'key': api_key}  # 高德地图API 获取Key
    parameters.update( {'keywords': kw,
                        'city':     cc,
                        'citylimit': True
                       } )

    pois = []   # 建构最终结果列表pois
    pg_no = 1   # 建构pg_no，每次给不一样的可选参数page值

    while True: # 不断迭代，直到 break跳出
        parameters.update({'page':pg_no})  #建构可选参数page值 为 pg_no

        r = requests.get ("http://restapi.amap.com/v3/place/text", params=parameters)
        data = r.json()

        pois.extend(data['pois'])         # 使用Python列表之extend方法，把返回数据中的结果data['pois']存入pois


        no_pois_this_search = len(pois)   # 计算 累积实质结果数
        no_pois = int(data['count'])      # 计算 应有的实质数量


        if (no_pois_this_search >= no_pois):  # 若 累积实质结果数 >= 应有的实质数量
            break                                   # 则结束迭代跳出
        else:
            pg_no +=1                               # 否则找下一页数据，把 pg_no增1


    # 使用pandas 模块处理数据
    df_input = pd.DataFrame(pois)

    # 选择部份栏位准备输出
    select_fields = "location	address	adname	cityname	id	importance	name	pname	tel	type	typecode".split("\t")
    df = df_input[select_fields]

    # 经纬度 增加新栏位字段
    df = df.assign( 经 = [x.split(',')[0] for x in df['location']])
    df = df.assign( 纬 = [x.split(',')[1] for x in df['location']])

    
    # 使用 return返回
    return(df)


## 使用函数

In [2]:
df_植物园_440000 = amap_api_search_by_keyword_city_code(kw='植物园', cc='440000', api_key='你自己申請到的key')
df_植物园_110000 = amap_api_search_by_keyword_city_code(kw='植物园', cc='110000', api_key='你自己申請到的key')
df_动物园_110000 = amap_api_search_by_keyword_city_code(kw='动物园', cc='110000', api_key='你自己申請到的key')


In [3]:
df_植物园_440000

,location,address,adname,cityname,id,importance,name,pname,tel,type,typecode,经,纬
0,"113.367196,23.181797",天源路1190号,天河区,广州市,B00140UEO5,[],华南植物园,广东省,020-85231993;020-85232037,风景名胜;公园广场;植物园,110103,113.367196,113.367196
1,"114.174956,22.578596",仙湖路160号,罗湖区,深圳市,B02F37T4NJ,[],深圳仙湖植物园,广东省,0755-25738430;0755-25722184,风景名胜;公园广场;植物园,110103,114.174956,114.174956
2,"113.52128,22.29402",健民路328号,香洲区,珠海市,B02F4000E1,[],农科奇观,广东省,0756-8535999;0756-8535888,风景名胜;风景名胜;国家级景点,110202,113.52128,113.52128
3,"110.275886,21.168908",湖秀新村1号(近湖光岩),麻章区,湛江市,B02F701056,[],南亚热带植物园,广东省,0759-2859145;12301,风景名胜;公园广场;植物园,110103,110.275886,110.275886
4,"113.620092,22.628194",新港大道,南沙区,广州市,B00140UFWZ,[],百万葵园,广东省,020-84956360;020-83309917;020-84956863,风景名胜;公园广场;公园,110101,113.620092,113.620092
5,"113.900516,22.500264",月亮湾青青路1号,南山区,深圳市,B02F30242B,[],月亮湾公园-青青世界,广东省,0755-26403988;0755-26646988;0755-26737330,风景名胜;风景名胜;风景名胜,110200,113.900516,113.900516
6,"113.770048,22.726385",和二一路32号,宝安区,深圳市,B02F37VJA4,[],海上田园-趣味植物园,广东省,[],风景名胜;风景名胜;风景名胜,110200,113.770048,113.770048
7,"116.756359,23.502012",广益街道官湖开发区,澄海区,汕头市,B02F2008RH,[],科隆千树园,广东省,0754-85858327;0754-86985976,风景名胜;公园广场;植物园,110103,116.756359,116.756359
8,"110.389479,21.251218",体育南路近体育馆,赤坎区,湛江市,B02F70NY85,[],南国热带花园,广东省,0759-3480019,风景名胜;公园广场;公园,110101,110.389479,110.389479
9,"110.409452,21.243933",海湾大桥旁,赤坎区,湛江市,B02F701S9V,[],中澳友谊花园,广东省,[],风景名胜;公园广场;公园,110101,110.409452,110.409452


In [4]:
df_植物园_110000 

,location,address,adname,cityname,id,importance,name,pname,tel,type,typecode,经,纬
0,"116.209714,40.001403",香山路,海淀区,北京市,B000A7783E,[],北京植物园,北京市,010-62591561;010-82598771,风景名胜;公园广场;植物园,110103,116.209714,116.209714
1,"116.352038,39.836168",南四环中路235号,丰台区,北京市,B000A7QDJF,[],世界花卉大观园,北京市,010-87500840;010-57283578,风景名胜;公园广场;植物园,110103,116.352038,116.352038
2,"116.796591,40.181584",杨镇鲜花港南路9号(白马路北侧),顺义区,北京市,B000A875HX,[],北京国际鲜花港,北京市,010-61417123;010-61417100;12301,风景名胜;风景名胜相关;旅游景点,110000,116.796591,116.796591
3,"116.594001,39.657730",104国道朱庄北口东行200米路北,大兴区,北京市,B0FFFFY2TD,[],呀路古热带植物园,北京市,010-80219731;010-80218180,风景名胜;公园广场;植物园,110103,116.594001,116.594001
4,"116.544787,39.634275",安采路路南,大兴区,北京市,B000A7UZO8,[],御林古桑园,北京市,010-80233242;010-80231151,风景名胜;公园广场;植物园,110103,116.544787,116.544787
5,"117.22074,40.658877",古北口镇汤河村,密云区,北京市,B000A9EU5V,[],紫海香堤香草艺术庄园,北京市,010-81052525,风景名胜;风景名胜;风景名胜,110200,117.22074,117.22074
6,"116.20211,40.010501",香山路北京植物园,海淀区,北京市,B000A7C5AJ,[],樱桃沟,北京市,[],风景名胜;公园广场;植物园,110103,116.20211,116.20211
7,"116.272583,40.034647",马连洼北路151号,海淀区,北京市,B000A84YM4,[],药用植物园,北京市,010-62899713,风景名胜;公园广场;植物园,110103,116.272583,116.272583
8,"116.435340,39.873576",左安门西街11号,东城区,北京市,B000A81GJX,[],龙潭西湖公园-北京教学植物园,北京市,[],风景名胜;公园广场;植物园,110103,116.435340,116.435340
9,"116.321106,40.007092",清华大学何添楼北侧,海淀区,北京市,B0FFF59171,[],清华大学花房,北京市,[],风景名胜;公园广场;植物园,110103,116.321106,116.321106


In [6]:
df_动物园_110000 

,location,address,adname,cityname,id,importance,name,pname,tel,type,typecode,经,纬
0,"116.336551,39.942131",西直门外大街137号,西城区,北京市,B000A87IUX,[],北京动物园,北京市,010-68314411;010-68390274;12301,风景名胜;公园广场;动物园,110102,116.336551,116.336551
1,"116.338812,39.938249",4号线/大兴线,西城区,北京市,BV10006771,[],动物园(地铁站),北京市,[],交通设施服务;地铁站;地铁站,150500,116.338812,116.338812
2,"116.335884,39.937965",27路;27路区间;87路;105路;107路;111路;332路;347路;360路;36...,西城区,北京市,BV10006770,[],动物园(公交站),北京市,[],交通设施服务;公交车站;公交车站相关,150700,116.335884,116.335884
3,"116.339837,39.947451",西直门外大街137号北京动物园,海淀区,北京市,B000A8358T,[],北京动物园(北门),北京市,[],通行设施;临街院门;临街院门,991400,116.339837,116.339837
4,"116.339892,39.941344",西城区,西城区,北京市,B0FFHOSWUM,[],动物园,北京市,[],地名地址信息;热点地名;热点地名,190700,116.339892,116.339892
5,"116.339559,39.938905",西直门外大街137号北京动物园,西城区,北京市,B000A834GY,[],北京动物园(南1门),北京市,[],通行设施;临街院门;临街院正门,991401,116.339559,116.339559
6,"116.330572,39.943419",五塔寺路,海淀区,北京市,B000A81K15,[],北京动物园(西北1门),北京市,[],通行设施;临街院门;临街院门,991400,116.330572,116.330572
7,"116.335187,39.944856",西直门外大街137号北京动物园,海淀区,北京市,B000A81FAS,[],北京动物园(西北3门),北京市,[],通行设施;临街院门;临街院门,991400,116.335187,116.335187
8,"116.328324,39.942265",西直门外大街137号北京动物园内,西城区,北京市,B000A18548,[],北京动物园(西北门),北京市,[],通行设施;临街院门;临街院门,991400,116.328324,116.328324
9,"116.333278,39.944097",五塔寺路22号,海淀区,北京市,B000A81FAR,[],北京动物园(西北2门),北京市,[],通行设施;临街院门;临街院门,991400,116.333278,116.333278
